### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

matchID homeTeamID awayTeamID leagueID round season  week day  \
33  42635839      79085      60678    51530     0      0  1136   5   
34  42636352       7826      79085        3     0      0  1136   6   
35  42637118       9514      79085    51530     0      0  1137   0   
36  42410331      89569      79085     1349     9     70  1137   1   
37  42640113      28394      79085    51530     0      0  1137   2   
38  42642298      79085      56459    51530     0      0  1137   3   
39  42410343      79085     124139     1349    10     70  1137   4   

        dateExpected       dateStarted homeTeamScore awayTeamScore supporters  \
33  2025-04-17 22:00  2025-04-17 22:00             0             0       2061   
34  2025-04-18 19:27  2025-04-18 19:28             5             0       5519   
35  2025-04-19 22:00  2025-04-19 22:00             0             1       4125   
36  2025-04-20 21:59  2025-04-20 21:59             0             1      21559   
37  2025-04-21 22:00  2025-04-21 22:00             0             0       7126   
38  2025-04-22 22:00  2025-04-22 22:00             0             0       2355   
39  2025-04-23 21:59  2025-04-23 21:59             1             3       9692   

   weather isFinished  
33       2          1  
34       2          1  
35       2          1  
36       2          1  
37       2          1  
38       2          1  
39       4          1

In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

7

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

teamID timeOnHalf timePossession offsides shoots fouls yellowCards  \
0   79085      41397          33501        2      4     3           1   
1   60678      28736          36632        0     12     2           1   
0    7826      12737          37003        2     21     1           1   
1   79085      57550          33284        0      0     0           0   
0    9514      23640          38201        1      7     1           0   
1   79085      46321          31760        5      5     2           1   
0   89569      44090          36837        3      2     1           0   
1   79085      26198          33451        0      9     3           0   
0   28394      24568          40514        0      6     1           0   
1   79085      45383          29437        1      2     0           0   
0   79085      39448          35614        4      5     4           3   
1   56459      30497          34331        7     13     1           0   
0   79085      40536          34796        1      5     5           2   
1  124139      29746          35486        3     13     1           0   

  redCards       tacticName ratingScoring ratingPassing ratingDefending  \
0        0           4-4-2b             9            10              11   
1        0              532             7             8              10   
0        0          5 3 2 o            10            10               8   
1        0           5-4-1a             6             7               8   
0        0  Concha Campions             9             9              10   
1        0           5-3-2c             9             9               9   
0        0          442 new             6             6              12   
1        0           5-3-2a            10             9               9   
0        0        433conAla            10            11              11   
1        0           5-4-1a             8             9              10   
0        1          3-4-3SP             9            10              10   
1        0               :)             9            10              10   
0        0           5-3-2a             9             9              10   
1        0          1-4-6-0             9             9               9   

    matchID  
0  42635839  
1  42635839  
0  42636352  
1  42636352  
0  42637118  
1  42637118  
0  42410331  
1  42410331  
0  42640113  
1  42640113  
0  42642298  
1  42642298  
0  42410343  
1  42410343

In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1700

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

teamID round points wins draws losses goalsScored goalsLost  \
0    10520    10     28    9     1      0          34         5   
1    23775    10     24    8     0      2          27        10   
2    23755    10     20    6     2      2          34        13   
3   124139    10     19    6     1      3          24        11   
4    78671    10     19    6     1      3          23        16   
5    79085    10     16    5     1      4          21        11   
6   122169    10     15    4     3      3          20        13   
7    79866    10     14    4     2      4          25        11   
8    89569    10      9    3     0      7          12        18   
9    79665    10      6    2     0      8          11        47   
10   80293    10      4    1     1      8           6        20   
11   89202    10      0    0     0     10           1        63   

         rankTotal  leagueID  
0   28529034009011      1349  
1   24517027008001      1349  
2   20521034006010      1349  
3   19513024006006      1349  
4   19507023006008      1349  
5   16510021005004      1349  
6   15507020004005      1349  
7   14514025004009      1349  
8    9494012003007      1349  
9    6464011002002      1349  
10   4486006001003      1349  
11    438001000012      1349

In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

Empty DataFrame
Columns: [leagueID, name, countryID, division, season, type, isOfficial, isCup]
Index: []

In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()